In [ ]:

import random #用于生成随机数
import copy #用于深拷贝对象
import re
import os
import numpy as np
import wandb
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence #用于填充序列以匹配长度
from transformers import AutoModelForCausalLM, AutoTokenizer #Hugging Face Transformers API，用于加载预训练语言模型和分词器
from datasets import load_dataset #Hugging Face Datasets API，用于加载数据集

def set_random_seed(seed: int = 42):
    random.seed(seed) #设置Python内置随机数生成器的种子
    torch.manual_seed(seed)#设置PyTorch的CPU随机数生成器种子
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)#设置所有GPU设备的随机数生成器种子
    torch.backends.cudnn.deterministic = True#启用确定性模式，确保cuDNN操作结果一致
    torch.backends.cudnn.benchmark = False#禁用自动优化算法选择

set_random_seed(42)

os.environ["WANDB_API_KEY"] = ""
os.environ["WANDB_PROJECT"] = ""

In [8]:
#定义系统提示模板，要求模型输出包含<reasoning>和<answer>标签的内容
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

#从模型生成的文本中提取<answer>标签内的内容
def extract_answer_from_model_output(text):
   parts = text.split("<answer>")#将文本按<answer>分割
   if len(parts) < 2:  # No <answer> tag found
       return None
   last_part = parts[-1] #提取最后一个<answer>标签后的内容
   if "</answer>" not in last_part:
       return None
   answer = last_part.split("</answer>")[0].strip() #提取<answer>和</answer>之间的内容
   return None if answer == "..." else answer

#从数据集中提取答案部分（以####分隔）
def extract_answer_from_dataset(text):
   
   # 检查文本是否包含'####'分隔符，该分隔符用于将问题与答案分开
   if "####" not in text:
       return None
   # 如果找到分隔符，则将文本在此分隔符处分割，并返回第二部分（答案）
   return text.split("####")[1].strip()

In [ ]:
from datasets import load_dataset
def prepare_dataset(split="train"):
   data = load_dataset('gsm8k', 'main')[split]
   formatted_data = []
   for example in data:
       prompt_str = build_prompt([
           {"role": "system", "content": SYSTEM_PROMPT},
           {"role": "user", "content": example["question"]}
       ]) #使用build_prompt函数生成提示字符串
       formatted_example = {
           "prompt": prompt_str,  # Now a string rather than a list.
           "answer": extract_answer_from_dataset(example["answer"])
       }
       formatted_data.append(formatted_example)
   return formatted_data

def build_prompt(messages):
   #消息列表转换为单个字符串提示
   return "\n".join([msg["content"].strip() for msg in messages])

def extract_last_number(text):
   #提取文本中最后出现的数字。
   text = text.replace('$', '').replace('%', '')
   #(?:^|\s|=) 匹配文本的开头、空白字符或等号 =
   #\s* 匹配零个或多个空白字符
   #(-?\d*\.?\d+) 匹配一个完整的数字（可以是整数或小数，支持负数）
   #\s*$ 确保匹配的数字出现在文本的末尾
   pattern = r'(?:^|\s|=)\s*(-?\d*\.?\d+)\s*$'
   match = re.search(pattern, text)
   #group(1) 提取正则表达式中第一个捕获组（即 (-?\d*\.?\d+)）的内容
   #将提取到的数字字符串转换为浮点数
   return float(match.group(1)) if match else None

def extract_single_number(text):
   #如果文本中只有一个数字，提取该数字
   #re.findall 是 Python 的正则表达式模块中的一个函数，用于查找所有匹配指定模式的子字符串
   #匹配一个完整的数字（可以是整数或小数，支持负数）
   numbers = re.findall(r'-?\d*\.?\d+', text)
   return float(numbers[0]) if len(numbers) == 1 else None

In [10]:
def evaluate_model(model, tokenizer, eval_examples, device):
   model.eval()
   correct = 0
   total = len(eval_examples)
   print("\n" + "="*50)
   print("EVALUATION ON", total, "EXAMPLES")
   print("="*50)
   for example in eval_examples:
       full_prompt = example["prompt"]
       expected = example["answer"]
       #使用 tokenizer.encode 将提示文本编码为模型输入张量，并将其移动到指定设备。
       inputs = tokenizer.encode(full_prompt, return_tensors="pt").to(device)
       #在 torch.no_grad() 上下文中运行模型推理，确保不计算梯度
       with torch.no_grad():
           outputs = model.generate(
               inputs,
               max_new_tokens=512,
               temperature=0.7,
               num_return_sequences=1,
               pad_token_id=tokenizer.pad_token_id,
               eos_token_id=tokenizer.eos_token_id,
               forced_eos_token_id=tokenizer.eos_token_id,
               early_stopping=False,
           )
        #使用 tokenizer.decode 将生成的 token 序列解码为字符串格式的响应
       response = tokenizer.decode(outputs[0], skip_special_tokens=True)
       try:
           predicted = extract_answer_from_model_output(response)
           if predicted == expected: 
               is_correct = True
           else:
               # Try single number matching
               pred_num = extract_single_number(str(predicted))
               exp_num = extract_single_number(str(expected))
               if pred_num is not None and exp_num is not None and pred_num == exp_num:
                   is_correct = True
               else:
                   # Try last number matching
                   pred_num = extract_last_number(str(predicted))
                   exp_num = extract_last_number(str(expected))
                   is_correct = (pred_num is not None and exp_num is not None and
                               pred_num == exp_num)
           # Update counter for correct answers
           if is_correct:
               correct += 1
           # Print evaluation details
           print("\nPrompt:")
           print(full_prompt)
           print("\nExpected Answer:")
           print(expected)
           print("\nExtracted Answer:")
           print(predicted)
           print("\nFull Generated Response:")
           print(response)
           print("\nCorrect:", "✓" if is_correct else "✗")
           print("-"*50)
       except Exception as e:
           print("\nFailed to parse model output for prompt:")
           print(full_prompt)
           print("Error:", e)
           print("-"*50)
   accuracy = (correct / total) * 100
   print(f"\nAccuracy: {accuracy:.2f}% ({correct}/{total})")
   print("="*50)
   #将模型恢复为训练模式（model.train()），以便后续继续训练
   model.train()
   return accuracy

#精确匹配
def correctness_reward(prompts, completions, answer, **kwargs): 
   #从 completions 中提取每个补全的文本内容
   responses = [completion[0]['content'] for completion in completions]
   extracted = [extract_answer_from_model_output(r) for r in responses]
   rewards = []
   for r, a in zip(extracted, answer):
       print("r:",r)
       print("a:",a)
       if r == a:  # Exact match case
           rewards.append(2.0)
       else:
           r_num = extract_single_number(str(r))
           a_num = extract_single_number(str(a))
           #如果两者都成功提取了数字，并且数字相等，认为是数值等价
           if r_num is not None and a_num is not None and r_num == a_num:
               rewards.append(1.5)
           else:
               rewards.append(0.0)
   #对每个补全文本调用 .split() 方法，将其按空格分割为单词列表。计算单词列表的长度，表示补全的长度
   completion_lengths = [len(response.split()) for response in responses]
   return rewards

#据模型生成内容是否符合指定的XML格式分配奖励分数
def format_reward(completions, **kwargs):
   responses = [completion[0]['content'] for completion in completions]
   rewards = []
   format_scores = []
   for response in responses:
       score = 0.0
       if "<reasoning>" in response: score += 0.2
       if "</reasoning>" in response: score += 0.2
       if "<answer>" in response: score += 0.2
       if "</answer>" in response: score += 0.2
       rewards.append(score)
       format_scores.append(score)
   return rewards

#将正确性奖励和格式奖励结合，生成综合奖励分数
def combined_reward(prompts, completions, answer):
   # Get individual rewards
   correctness_scores = correctness_reward(prompts=prompts, completions=completions, answer=answer)
   format_scores = format_reward(completions=completions)
   # Combine rewards - correctness is weighted more heavily
   combined_rewards = []
   for c_score, f_score in zip(correctness_scores, format_scores):
       # Correctness score range: 0.0 to 2.0
       # Format score range: 0.0 to 0.8
       # Total range: 0.0 to 2.8
       combined_rewards.append(c_score + f_score)
   return combined_rewards

#计算特定 token 的 log 概率
def selective_log_softmax(logits, input_ids):
    #logits（torch.Tensor）：模型输出的原始 logits，形状通常为 [batch_size, sequence_length, vocab_size]
    #input_ids（torch.Tensor）：需要计算 log 概率的 token ID，形状通常为 [batch_size, sequence_length]
    
    #dim=-1 表示在最后一个维度（词汇表维度）上进行操作
    #结果是一个形状与 logits 相同的张量，表示每个 token 在词汇表上的 log 概率
    #将 input_ids 的形状从 [batch_size, sequence_length] 扩展为 [batch_size, sequence_length, 1]
    #使用 gather 操作从 log_probs 中提取指定位置的值
    #去掉最后一个多余的维度，将结果形状恢复为 [batch_size, sequence_length]
    log_probs = nn.functional.log_softmax(logits, dim=-1)
    return log_probs.gather(dim=-1, index=input_ids.unsqueeze(-1)).squeeze(-1)

#计算一批 token 的 log 概率
def compute_log_probs(model, input_ids, attention_mask, logits_to_keep):
    #logits_to_keep（int）：从序列末尾保留的 token 数量
    
    #logits[:, :-1, :]选择所有 token 的 logits，除了最后一个 token
    #在语言建模任务中，目标是预测下一个 token，因此每个位置的 logits 对应于下一个 token 的概率分布
    #结果形状：[batch_size, sequence_length - 1, vocab_size]
    logits = model(input_ids=input_ids, attention_mask=attention_mask).logits[:, :-1, :]
    #从 input_ids 和 logits 中截取最后 logits_to_keep 个 token
    #结果形状：[batch_size, logits_to_keep]
    input_ids = input_ids[:, -logits_to_keep:]
    #结果形状：[batch_size, logits_to_keep, vocab_size]
    logits = logits[:, -logits_to_keep:, :]
    return selective_log_softmax(logits, input_ids)

#为生成的补全文本创建一个掩码，忽略序列中 EOS（End-of-Sequence）标记之后的所有 token
def create_completion_mask(completion_ids, eos_token_id): 
    #对 completion_ids 中的每个元素进行比较，判断是否等于 eos_token_id
    #返回一个布尔张量 is_eos，形状与 completion_ids 相同，其中值为 True 表示该位置是 EOS 标记
    is_eos = completion_ids == eos_token_id
    #创建一个形状为 [batch_size] 的张量，初始值为 is_eos.size(1)（即序列的最大长度）
    #数据类型为 torch.long，设备与 completion_ids 一致
    eos_idx = torch.full((is_eos.size(0),), is_eos.size(1), dtype=torch.long, device=completion_ids.device)
    #按行（dim=1）检查 is_eos 中是否存在至少一个 True 值
    mask_exists = is_eos.any(dim=1)
    #将布尔张量 is_eos 转换为整数张量
    #按行（dim=1）找到第一个最大值（即第一个 1）的索引
    #仅对包含 EOS 标记的序列更新 eos_idx
    eos_idx[mask_exists] = is_eos.int().argmax(dim=1)[mask_exists]
    #创建一个从 0 到 sequence_length - 1 的范围张量
    #将范围张量扩展为形状 [batch_size, sequence_length]，以便与 completion_ids 匹配
    sequence_indices = torch.arange(is_eos.size(1), device=completion_ids.device).expand(is_eos.size(0), -1)
    #将 eos_idx 的形状从 [batch_size] 扩展为 [batch_size, 1]，以便与 sequence_indices 进行广播操作
    #比较每个 token 的索引是否小于或等于第一个 EOS 标记的索引
    #将布尔张量转换为整数张量
    return (sequence_indices <= eos_idx.unsqueeze(1)).int()

def generate_completions(model, tokenizer, prompts, num_generations=4, max_completion_length=32):   
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #使用分词器将 prompts 编码为 PyTorch 张量
    #padding=True：对序列进行填充，使其长度一致
    #padding_side="left"：在左侧填充，确保补全生成从提示的末尾开始
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, padding_side="left")
    prompt_ids = inputs["input_ids"].to(device)
    prompt_mask = inputs["attention_mask"].to(device)
    print(f"Input batch size: {prompt_ids.size(0)}, Device before model: {prompt_ids.device}")
    #获取提示的序列长度
    prompt_length = prompt_ids.size(1)
    #在批次维度上重复每个提示 num_generations 次
    prompt_ids = prompt_ids.repeat_interleave(num_generations, dim=0)
    prompt_mask = prompt_mask.repeat_interleave(num_generations, dim=0)
    outputs = model.generate(
        prompt_ids,
        attention_mask=prompt_mask,
        max_new_tokens=max_completion_length,
        do_sample=True,#启用采样生成（而非贪婪解码）
        temperature=1.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        early_stopping=False
    )
    print(f"Output batch size: {outputs.size(0)}, Device after model: {outputs.device}")
    #截取生成输出中的补全文本部分（排除提示部分）
    completion_ids = outputs[:, prompt_length:]
    #调用 create_completion_mask 函数，生成补全掩码，忽略 EOS 标记之后的 token
    completion_mask = create_completion_mask(completion_ids, tokenizer.eos_token_id)
    return prompt_ids, prompt_mask, completion_ids, completion_mask

#生成用于 GRPO（Generalized Reinforcement Policy Optimization）训练所需的数据，包括补全文本、log 概率等
def generate_rollout_data(model, ref_model, tokenizer, batch_samples, num_generations, max_completion_length):
    #model：当前策略模型，用于生成补全和计算 log 概率
    #ref_model：参考模型，用于计算 KL 散度
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #遍历 batch_samples，如果样本是字典，则提取 "prompt" 字段；否则提取第一个元素
    prompts = [sample["prompt"] if isinstance(sample, dict) else sample[0] for sample in batch_samples]
    answers = [sample["answer"] if isinstance(sample, dict) else sample[1] for sample in batch_samples]
    with torch.no_grad():
        prompt_ids, prompt_mask, completion_ids, completion_mask = generate_completions(
            model, tokenizer, prompts, num_generations, max_completion_length
        )
        #在序列维度上拼接提示和补全的 token ID
        input_ids = torch.cat([prompt_ids, completion_ids], dim=1)
        attention_mask = torch.cat([prompt_mask, completion_mask], dim=1)
        #获取补全文本的长度（token 数量），用于指定需要保留的 logits 数量
        logits_to_keep = completion_ids.size(1)
        old_log_probs = compute_log_probs(model, input_ids, attention_mask, logits_to_keep)
        ref_log_probs = compute_log_probs(ref_model, input_ids, attention_mask, logits_to_keep)
    #遍历 completion_ids，对每个补全调用 tokenizer.decode 将其解码为字符串
    formatted_completions = [[{'content': tokenizer.decode(ids, skip_special_tokens=True)}] for ids in completion_ids]
    #对每个提示重复 num_generations 次
    repeated_prompts = [p for p in prompts for _ in range(num_generations)]
    repeated_answers = [a for a in answers for _ in range(num_generations)]
    # input_ids：完整的输入 token ID（提示 + 补全）。
    # attention_mask：完整的注意力掩码（提示 + 补全）。
    # completion_mask：补全文本的掩码。
    # old_log_probs：策略模型的 log 概率。
    # ref_log_probs：参考模型的 log 概率。
    # formatted_completions：格式化的补全文本。
    # repeated_prompts：重复的提示。
    # repeated_answers：重复的答案。
    # logits_to_keep：需要保留的 logits 数量。
    # batch_size：批次大小（提示数量）。
    # num_generations：每个提示生成的补全数量
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "completion_mask": completion_mask,
        "old_log_probs": old_log_probs,
        "ref_log_probs": ref_log_probs,
        "formatted_completions": formatted_completions,
        "repeated_prompts": repeated_prompts,
        "repeated_answers": repeated_answers,
        "logits_to_keep": logits_to_keep,
        "batch_size": len(prompts),
        "num_generations": num_generations
    }

In [11]:
#计算用于更新策略模型的 GRPO（Generalized Reinforcement Policy Optimization）损失
def grpo_loss(model, ref_model, rollout_data, tokenizer, reward_function, beta=0.01, epsilon=0.2):
    # model：当前策略模型，用于生成补全文本和计算 log 概率。
    # ref_model：参考模型，用于计算 KL 散度。
    # rollout_data（dict）：由 generate_rollout_data 函数生成的数据，包含输入 token ID、注意力掩码、补全掩码、旧的 log 概率等。
    # tokenizer：分词器，用于对文本进行编码和解码。
    # reward_function：奖励函数，用于计算每个补全的奖励分数。
    # beta（float）：KL 散度惩罚系数，默认为 0.01。
    # epsilon（float）：PPO 裁剪参数，默认为 0.2。  
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    input_ids = rollout_data["input_ids"]
    attention_mask = rollout_data["attention_mask"]
    completion_mask = rollout_data["completion_mask"]
    logits_to_keep = rollout_data["logits_to_keep"]
    old_log_probs = rollout_data["old_log_probs"]
    ref_log_probs = rollout_data["ref_log_probs"]
    #使用当前策略模型计算补全文本的 log 概率
    token_log_probs = compute_log_probs(model, input_ids, attention_mask, logits_to_keep)
    # token_log_probs - old_log_probs：计算新旧策略的 log 概率差。
    # torch.exp(...)：将 log 概率差转换为概率比
    ratio = torch.exp(token_log_probs - old_log_probs)
    #输入提示、格式化的补全文本和答案，计算奖励分数
    rewards = torch.tensor(
        reward_function(prompts=rollout_data["repeated_prompts"], completions=rollout_data["formatted_completions"], answer=rollout_data["repeated_answers"]),
        dtype=torch.float32,
        device=device
    )
    # print(f"Rewards: {rewards}")  # Debug rewards
    batch_size = rollout_data["batch_size"]
    num_generations = rollout_data["num_generations"]
    #：将奖励分数重新组织为 [batch_size, num_generations] 的形状。
    rewards = rewards.view(batch_size, num_generations)
    # avg_reward：计算所有奖励的平均值，并打印出来。
    # mean_rewards：计算每个提示的奖励均值，并重复以匹配补全数量。
    # std_rewards：计算每个提示的奖励标准差，并重复以匹配补全数量。
    # advantages：通过标准化公式计算优势函数，并调整形状
    avg_reward = rewards.mean().item()
    print("Average Reward:", avg_reward)
    mean_rewards = rewards.mean(dim=1).repeat_interleave(num_generations)
    std_rewards = rewards.std(dim=1).repeat_interleave(num_generations)
    advantages = ((rewards.view(-1) - mean_rewards) / (std_rewards + 1e-4)).unsqueeze(1)
    #计算 PPO 替代目标
    #计算未裁剪的目标
    surr1 = ratio * advantages
    #对概率比进行裁剪，限制在 [1 - epsilon, 1 + epsilon] 范围内
    #计算裁剪后的目标
    surr2 = torch.clamp(ratio, 1 - epsilon, 1 + epsilon) * advantages
    surrogate_loss = torch.min(surr1, surr2)
    #计算 KL 散度
    kl = torch.exp(ref_log_probs - token_log_probs) - (ref_log_probs - token_log_probs) - 1
    #组合替代损失和 KL 散度，得到每个 token 的损失
    per_token_loss = surrogate_loss - beta * kl
    #(per_token_loss * completion_mask).sum(dim=1)：按补全掩码加权求和
    #/ completion_mask.sum(dim=1)：对每个补全的损失进行归一化
    loss = -((per_token_loss * completion_mask).sum(dim=1) / completion_mask.sum(dim=1)).mean()
    return loss, avg_reward

#使用 GRPO（Generalized Reinforcement Policy Optimization）方法对语言模型进行训练
def train_with_grpo(model, tokenizer, train_data, num_iterations=1, num_steps=500, batch_size=4,
                              num_generations=4, max_completion_length=128, beta=0.1,
                              learning_rate=5e-6, mu=3, epsilon=0.2, reward_function=None, device_ids=None):
    # beta（float）：KL 散度惩罚系数。
    # learning_rate（float）：优化器的学习率。
    # mu（int）：每个批次中的策略更新次数。
    # epsilon（float）：PPO 裁剪参数。
    # reward_function：奖励函数，用于计算补全的奖励分数
 
    #确保代码在至少两个 GPU 上运行，并设置默认设备
    assert device_ids is not None and len(device_ids) > 1, "This code needs at least 2 GPU cores to run!"
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #nn.DataParallel(model, device_ids=device_ids)：将模型分配到指定的 GPU 上
    model = nn.DataParallel(model, device_ids=device_ids)
    print(f"Model wrapped with DataParallel across GPUs: {device_ids}")
    #执行多次外层迭代，每次迭代都会创建一个新的参考模型并重新初始化优化器
    for iteration in range(num_iterations):
        print(f"\nIteration {iteration+1}/{num_iterations}")
        #复制当前策略模型（model.module 是 DataParallel 包装的原始模型）为参考模型
        ref_model = copy.deepcopy(model.module)
        ref_model.eval()
        for param in ref_model.parameters():
            param.requires_grad = False #冻结参考模型的参数，避免梯度更新
        print("Reference model created.")
        #为当前策略模型重新初始化优化器，并将其设置为训练模式
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
        model.train()
        #每个训练步骤中，随机采样一批数据并生成补全文本及其相关数据
        for step in range(num_steps):
            batch_samples = random.sample(train_data, batch_size)
            with torch.no_grad():
                rollout_data = generate_rollout_data(
                    model.module,
                    ref_model,
                    tokenizer,
                    batch_samples,
                    num_generations,
                    max_completion_length
                )
            for grpo_iter in range(mu):
                loss, avg_reward = grpo_loss(
                    model.module,
                    ref_model,
                    rollout_data,
                    tokenizer,
                    reward_function,
                    beta=beta,
                    epsilon=epsilon
                )
                # 调用 grpo_loss 函数，计算 GRPO 损失和平均奖励。
                # optimizer.zero_grad()：清空梯度。
                # loss.backward()：反向传播计算梯度。
                # torch.nn.utils.clip_grad_norm_(...)：裁剪梯度，防止梯度爆炸。
                # optimizer.step()：更新模型参数
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)
                optimizer.step()
                # Log to wandb
                wandb.log({
                    "loss": loss.item(),
                    "average_reward": avg_reward,
                    "iteration": iteration + 1,
                    "step": step + 1,
                    "grpo_iter": grpo_iter + 1
                })
                print(f"Iteration {iteration+1}/{num_iterations}, Step {step+1}/{num_steps}, "
                      f"GRPO iter {grpo_iter+1}/{mu}, loss: {loss.item():.4f}")
                #for i in range(torch.cuda.device_count()):
                #    print(f"GPU {i} Usage: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MiB, "
                #          f"Utilization: {torch.cuda.utilization(i)}%")
                # Uncomment to see the GPU utilization stats
    return model.module

In [ ]:
#优化模型以减少训练过程中的内存占用
def optimize_model_memory(model): 
    model.train()
    #禁用键值（KV）缓存以节省内存
    # 在生成任务中，KV 缓存用于存储注意力机制中的中间结果以加速推理。但在训练过程中，禁用 KV 缓存可以显著减少内存占用。
    model.config.use_cache = False
    # First ensure inputs will require gradients
    #确保输入嵌入层的输出需要梯度，以便在反向传播时计算梯度
    #检查模型是否具有内置方法 enable_input_require_grads
    if hasattr(model, "enable_input_require_grads"):
        model.enable_input_require_grads()
    else:
        #定义一个前向钩子函数 make_inputs_require_grad，将输入嵌入层的输出标记为需要梯度
        def make_inputs_require_grad(module, input, output):
            output.requires_grad_(True)
        #使用 register_forward_hook 将钩子函数注册到输入嵌入层
        model.get_input_embeddings().register_forward_hook(make_inputs_require_grad)
    #启用梯度检查点（Gradient Checkpointing），以牺牲计算效率换取内存节省
    #梯度检查点是一种内存优化技术，通过重新计算部分前向传播的结果来减少内存占用。
    #这在训练大型语言模型时非常有用，尤其是在 GPU 内存有限的情况下
    model.gradient_checkpointing_enable()
    return model

# Main execution
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using primary device: {device}")

model_name = "Qwen2.5-3B-Instruct"
output_dir = "math_solver_model"
print("Downloading model...")
# AutoModelForCausalLM.from_pretrained(...)：
# 加载因果语言模型（Causal Language Model）。
# torch_dtype=torch.bfloat16：使用 bfloat16 数据类型以节省显存。
# device_map="auto"：自动将模型分配到多个 GPU 或 CPU 上
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
print("Model downloaded")


#padding_side="left"：设置填充方向为左侧，确保补全生成从提示的末尾开始
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
#tokenizer.pad_token = tokenizer.eos_token：将填充 token 设置为结束 token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
model.config.eos_token_id = tokenizer.eos_token_id

num_gpus = torch.cuda.device_count()
print(f"Detected {num_gpus} GPUs")
device_ids = list(range(num_gpus)) if num_gpus > 1 else None
all_data = prepare_dataset("train")
random.shuffle(all_data)
size_of_eval_data = 30  # change to a smaller value to save time or to a larger number for a more reliable estimate
eval_data = all_data[:size_of_eval_data]
train_data = all_data[size_of_eval_data:]

print("\nInitial model evaluation before finetuning:")
pre_grpo_accuracy = evaluate_model(model, tokenizer, eval_data, device)
print(f"Pre-GRPO Accuracy: {pre_grpo_accuracy:.2f}%")

model = optimize_model_memory(model)

print("\nStarting RL fine-tuning using GRPO...")
# This config was tested on a 8xA100 node, where each A100 is has 80GB of VRAM
training_config = {
    'num_iterations': 1,
    'num_steps': 500,
    'batch_size': 7, # reduce if you have fewer GPUs
    'num_generations': 8, # reduce if you have GPUs with less VRAM
    'max_completion_length': 256, # reduce if you have GPUs with less VRAM
    'beta': 0.04,
    'learning_rate': 5e-6,
    'mu': 1,
    'epsilon': 0.1
}

# Initialize Weights & Biases
wandb.init(project=os.environ["WANDB_PROJECT"], reinit=True)
print("Weights & Biases initialized.")

model = train_with_grpo(
    model=model,
    tokenizer=tokenizer,
    train_data=train_data,
    reward_function=combined_reward,
    device_ids=device_ids,
    **training_config
)

wandb.finish()
print("Training completed and wandb run finished.")

print("\nFinal model evaluation after GRPO RL fine-tuning:")
post_grpo_accuracy = evaluate_model(model, tokenizer, eval_data, device)
print(f"Post-GRPO Accuracy: {post_grpo_accuracy:.2f}%")

print("\nSaving GRPO fine-tuned model...")
# model.save_pretrained(...)：保存微调后的模型权重。
# tokenizer.save_pretrained(...)：保存分词器配置
model.save_pretrained("grpo_finetuned_model")
tokenizer.save_pretrained("grpo_finetuned_model")